<h1 style="color:purple;"><center><big>B92 Protocol</big></center></h1>
<h4><center>By Graeme Jacobson</center></h4>

<h1 style="color:Fuchsia">What is Quantum Cryptography?</h1>

Cybersecurity is one of the most important aspects of any organization in the modern era. It protects sensitive information for all kinds of people in an organization. Whether it is your banking information, social media credentials, or email, it is all protected by an important aspect of cybersecurity: Cryptography. Cryptography is the method of using math to disguise information and make it almost impossible to undisguise. These methods for protecting our information are done behind the scenes, making the everyday person feel confident online, without needing to be a math or computer science wizard.

As quantum computing and quantum photonics become more advanced, one of the many pratical applications will be in Cryptography. As it turns out, the  <a href="https://en.wikipedia.org/wiki/No-cloning_theorem">Quantum No-Cloning Theorem</a> creates very nice security blanket for quantum cryptography protocols. This theorem states that you cannot create a copy of an arbitrary, unknown quantum state. That is, if you were to receive the $|+\rangle$ state, you cannot create a copy of this state, because you would need to measure the state, collapsing the state to either $|0\rangle$ or $|1\rangle$. The only way to copy the state would be if the person that sent you the $|+\rangle$ state, told you what it was, allowing you to prepare another $|+\rangle$ state (but you don't want to disclose what that original state is).

<a href="https://en.wikipedia.org/wiki/Quantum_key_distribution">Quantum Key Distribution</a> (QKD) is the way that we take advantage of the no-cloning theorem to send ecrypted messages. Essentially, one party will send a number of quantum states to the other party, then will simulatenously create a private key and check if their quantum communication channel has an eavesdropper present. They will then decide on the final private key, and finally encode their message with said key. Since quantum states cannot be cloned, an eavesdropper will not be able to copy the states sent through the quantum channel between the two partie. Irregularities in the states being sent will be considered corrupted, and these states will be thrown out. That is really amazing because classically, you cannot tell if someone else is in your communication channel, but now, you can tell if someone is looking at your information. That is why quantum key distribution is considered one of the most promising forms of cybersecurity in the future.

<p style="text-align:center"><img src="/filesystem/home/graemejacobson-40qbraid-2ecom/personal/images/b92_overview.png"></p>

<h1 style="color: Fuchsia">B92 Protocol</h1>

The B92 Protocol is one of the most popular forms of quantum key distribution, behind the BB84 protocol, that does not use quantum entanglement (that's a story for another day). This protocol uses two non-orthogonal bases, for example, the $X$ and $Z$ bases. It is known to be more secure than BB84 because of that fact, meaning less final states are accepted. However, that also means that it is not a very efficient method since a majority of quantum states that are sent will be thrown out. Typically around $25\%$ of states are kept for the final key, meaning that if you wanted to have a 25 bit key, you would need to send around 100 quantum states.

Let's say Alice wants to send Bob an ecrypted message. For the sake of this blog post, I will say that the two non-orthogonal bases that will be used are the $X$ and $Z$ bases.

*Note that the basis states for $X$ are:

$|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$, $|-\rangle = \frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$

*The basis states for $Z$ are just $|0\rangle$ and $|1\rangle$

<p style="text-align:center"><img src="/filesystem/home/graemejacobson-40qbraid-2ecom/personal/images/b92_basis.png"></p>

To start, Alice will generate a random bit string of $0$ 's and $1$ 's (size depends on how long of a message she wants to send). A $0$ will correspond to a $|0\rangle$, and a $1$ will correspond to a $|+\rangle$. Alice will then send all these states to Bob via a quantum channel. Bob will now generate his own random bit string, where $0$ will correspond to measuring a state in the $Z$ basis, and $1$ corresponds to measuring in the $X$ basis. Once Bob has measured each state using his basis choices, he will communicate to Alice over a classical channel which positions in the string of states he measured to be $|1\rangle$ ( $|1\rangle$ is the least frequent measurement and can still be measured from either original state $|0\rangle$ or $|+\rangle$). Finally, Alice will keep her original bits at the positions where $|1\rangle$ was measured, and Bob will keep the opposite of his original bit string at those positions (so it matches Alice's final string). Now, they have the private key made to encode information for a classical channel.

However, we typically take one last security measure, and pick a few positions from the private key and discuss what they are over the classical channel. Naturally, these positions will be deleted since they were publically dicussed, but this part of the process can detect if there was an eavesdropper in the quantum channel. If the positions they dicussed were the same (Bob's key being exactly opposite of Alice's), things are all good. If there is no tampering with the quantum states, eavesdroppers or errors, the bits should match. If any of the bits do not match, they will throw away the entire private key, assuming there was an eavesdropper, and start the process over again.

Look at the example chart below. You can see Alice's bit string denoted by "a bit" and Bob's bit string denoted by "a' bit". After Alice sent her states to Bob and Bob measured each state, you can see Bob's measurements and the percentage of the time he would measure that particular state in that system. Notice how you never measure $|1\rangle$ with $100\%$ chance. This is on purpose, so that the least amount of states are finally selected, increasing security of the protocol. Finally, noticed the three positions where a $|1\rangle$ was measured. Alice's bit was chosen as the bit for the private key, 010, and Bob's bit should be the exact opposite of this bit, 101. Bob will flip all his bits, so that him and Alice share the same key. If Bob flips these bits and gets something other than Alice's key, we know there was an eavesdropper in the channel.

<p style="text-align:center"><img src="/filesystem/home/graemejacobson-40qbraid-2ecom/personal/images/b92_chart.png"></p>

<h1 style="color:Fuchsia">B92 Implementation</h1>

Now, let's go through the step by step process again, and use some python to show off it's beauty. I'll show both an ideal case and the case where an eavesdropper is present. Again, the final goal here is to create a private key between Alice and Bob, so they can later send each other encrypted messages.

**Keep in mind, this is going to be a very dumbed down version of the real process, and normally the bit strings would be massive to send long messages. I am also not going to encode messages, and only show the process of QKD.

In [1]:
from qiskit import *
from numpy import pi
import random
import string

Before we actually start the B92 protocol, I am going to make two functions that will come into play throughout the protocol. The first, generate_bit_string, will build a random bit string from the choices of bit_choices for a specified length. The second, generate_qc, will build onto quantum circuit, qc, based on a bit_string. For B92, a Hadamard Gate will be place on the qubit corresponding to the position in the bit string that reads a $1$.

In [2]:
def generate_bit_string(bit_choices, length):
    
    bit_string = []
    
    for i in range(length):
        curr_bit = random.choice(bit_choices)
        bit_string.append(curr_bit)
        
    return bit_string


def generate_qc(bit_string, qc):
    
    for qubit in range(len(bit_string)):
        if bit_string[qubit] == 1:
            qc.h(qubit)
        else:
            continue
        
    qc.barrier()

Like before, we will begin with Alice randomly generating her bit string. Note that just like with classical bits, her selection can either be a $0$ or a $1$. I am, also, going to set the length of our bit_string, here, to a length of $20$. Like I mentioned before, normally, these strings will be much larger than this.

In [3]:
bit_choices = [0,1]
bit_length = 20

Now, let's generate our bit_string for Alice using the function we made earlier.

In [4]:
alice_string = generate_bit_string(bit_choices, bit_length)
print(alice_string)

Now, we will initalize our quantum circuit, so we can build the initial states that Alice will be sending to Bob. Note that I am making both a QuantumRegister and ClassicalRegister. This is because we will be measuring all these qubits in order to securely create a private key.

In [5]:
qr = QuantumRegister(bit_length)
cr = ClassicalRegister(bit_length)
qc = QuantumCircuit(qr, cr)

qc.draw()

At this point, everything should be setup to continue with the protocol. So, we can finally build our states that Alice will be sending to Bob based on her bit string. Remember, whenever her bit string has a $1$, we will place an $H$ gate, to make it a $|+\rangle$ state. Otherwise, we leave the state as $|0\rangle$.

In [6]:
generate_qc(alice_string, qc)
qc.draw()

Sweet! We have our quantum states for Alice. You can see by the circuit where Alice had a $1$ or $0$ in her bit string! She will send these states down the quantum channel to Bob, where he will make his measurements. But first, Bob needs to decide how he's going to measure each of theses quantum states.

Remember, once these states leave Alice, you must measure the state in order to see what it is. So, think about anywhere beyond the barrier that I placed on the circuit, you will not know the state unless you measure it.

Now, let's generate Bob's bit string.

In [7]:
bob_string = generate_bit_string(bit_choices, bit_length)
print(bob_string)

Bob now has his bit string. Like when Alice used the bit string to decide what state she sent, Bob will use his bit string to decide what basis to measure the qubit in. You can measure in the $Z$ basis by not placing any gates right before the measurement. You can measure in the $X$ basis by placing an $H$ gate before the measurement.

In [8]:
generate_qc(bob_string, qc)
qc.draw()

Bob's gates are now placed in the circuit to decide how he will measure each qubit. I have not measured these qubits just yet. Take a moment to guess what will happen if a certain qubit has either no gates on it, or has two $H$ gates on it. What should Bob measure?

If you guessed $|0\rangle$, you are correct! If the qubit is measured in the basis that it is currently in, it will measure a $|0\rangle$. This means the only way to measure a $|1\rangle$ is if only one of Alice or Bob placed an $H$ gate on the circuit. Even then, this will measure $|1\rangle$  $50\%$ of the time. This means that you should measure $|1\rangle$ about $25\%$ of the time in total, and $|0\rangle$ $75\%$ of the time. That won't always be spot on due to the randomized nature of quantum, but it's a good estimate.

Let's see how our measurements come out for Bob. Start by measuring all the qubits, then we will use IBM's qasm-simulator to run the results.

In [9]:
qc.measure(qr, cr)
qc.draw()

In [10]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1, memory=True)
results = job.result().get_memory()

result_string = results[0] #get the string of bit measurements
result_split = list(map(lambda i:i, result_string)) #split the string of measurements
key = list(map(int, result_split)) #turn each bit into an int
key.reverse() #for correct order in alice bits
print(key)

And there you have it, our key. Like I commented in the code, qiskit measurement results read from right to left, so I reversed the string in order to have it read from left to right. Note, as well, the amount of $1$'s that came out of the measurements. It should be around five bits, which is about $25\%$ of the key.

However, this is not yet the final key. We need to first, find the bits for only the positions that have a $1$, and then perform our security measure on the key.

Let's make our private key by using Alice's bit string at the positions where our 'key' above is a $1$. Remember, Bob will use this same procedure to get his key, but he should have the opposite bit at each position. So, the opposite of his string will be Alice's string, which ends up being the private key.

In [11]:
private_key_alice = []
private_key_bob = []

for i in range(len(key)):
    if key[i] == 1:
        private_key_alice.append(alice_string[i])
        private_key_bob.append(bob_string[i])
    else:
        continue
        
print(private_key_alice)
print(private_key_bob)

Just like predicted, their keys are exactly the opposite. However, we use Alice's string as the private key. For Bob, that will just be the opposite of his string. 

Now, that we have our key, Alice and Bob will do a security measure to make sure that there was no eavesdropper present. To do this, they will share random parts of their key and make sure that they have the same bits (once Bob has flipped all the bits of his private key). If they do not, that will assume that their quantum channel was comprimised, and they will throw out the entire key and start again. Otherwise, they will just throw out the keys that they checked and keep the rest as the final private key for encryption.

To make more sense of why this security measure works, let's show what this protocol might look like if there is an eavesdropper present.

<br></br>

Starting from the top again, let's say that Alice's bit string is $1011000101$. She will make her quantum states.

**I decreased the size for this example

In [27]:
alice_string_e = [1,0,1,1,0,0,0,1,0,1]

qre = QuantumRegister(10)
cre = ClassicalRegister(10)
qce = QuantumCircuit(qre, cre)

generate_qc(alice_string_e, qce)
qce.draw()

Alice has now sent these states down the quantum channel. But, an eavesdropper is present on this channel. Let's call her Eve. In order for Eve to see the states, she will have to measure the states. Remember, in quantum mechanics, if you measure a quantum state, you collapse the entire state, and all prior information about that state is lost.

In [28]:
qce.measure(qre, cre)
qce.barrier()
qce.draw()

Eve will go ahead and build her bit string in the same fashion, so she can examine what states she has gotten.

In [29]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(qce, simulator, shots=1, memory=True)
resultse = job.result().get_memory()

result_stringe = resultse[0] 
result_splite = list(map(lambda i:i, result_stringe)) 
keye = list(map(int, result_splite)) 
keye.reverse() 
print(keye)

Let's take a look at what the eavesdropper measured. Notice, she cannot tell if a state was a $|0\rangle$ or $|+\rangle$ originally. She can only make an inference. A good assumption would be to place an $H$ gate where a $|1\rangle$ was measured, but remember, some of those $|0\rangle$'s also came from an $H$ gate. So, Eve will have to guess the initial states. 

That is the power of the no-cloning theorom. Quantum states cannot be cloned, so Eve will have to guess what state she was sent. These guess will be sent onto Bob, where the protocol will resume.

Eve will now place $H$ gates on qubits with her best guestimate.

In [30]:
qce.h(0)
qce.h(2)
qce.h(6)
qce.h(8)
qce.barrier()

qce.draw()

Notice how the eavesdropper can guess where the $H$ gate will go, where a $|1\rangle$ is measured (though I am running a new circuit each time, so it will not be perfect), but is guessing for all of the $|0\rangle$ states that were measured. So, some of these guesses are going to be wrong.

After the gates are placed and new states are formed, Eve will send these down the quantum channel to Bob, acting like she wasn't even there.

Bob will go on to continue by measuring these states. I will make a random bit string for Bob to use on his measurements.

In [31]:
bob_string_e = [1,1,0,0,1,0,0,0,1,0]

generate_qc(bob_string_e, qce)
qce.measure(qre, cre)
qce.draw()

In [32]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(qce, simulator, shots=1, memory=True)
results = job.result().get_memory()

result_string = results[0] 
result_split = list(map(lambda i:i, result_string)) 
key = list(map(int, result_split)) 
key.reverse() 
print(key)

So, Bob made his measurements, and him and Alice have discussed which positions were measured to be a $|1\rangle$. Below, I am going to do out the process for both. The first time we did this, using an ideal case, Alice and Bob's private string were exact opposites of each other. However, since there was an eavesdropper present in this case, that should not be the case.

Let's create the private strings for Alice and Bob just like we did previously and compare them.

In [33]:
private_key_a = []
private_key_b = []

for i in range(len(key)):
    if key[i] == 1:
        private_key_a.append(alice_string_e[i])
        private_key_b.append(bob_string_e[i])
    else:
        continue
        
print(private_key_a)
print(private_key_b)

Look at that! Their strings are not exactly opposite of each other. If Alice and Bob were to compare their bits, they would notice an inconsistency. They will now assume that their quantum channel was comprimised and restart the process over again.

The power of the no-cloning theorom and quantum measurements allowed for Alice and Bob to see that there was an eavesdropper in their channel. This is not something that is possible classically, and gives quantum cryptography a nice advantage over its classical analogue.